<a href="https://colab.research.google.com/github/seanlee9959/amazon-NLP/blob/main/amzn_webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Webscraper.

In [ ]:
#%% Code


# Import packages
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions
import pandas as pd
import time

# Create chrome instance and navigate to website
options = ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(executable_path='C:/Users/cjconst/Documents/ECON 370/chromedriver.exe', options=options)
search = "microwave"
url_template = "https://www.amazon.com/s?k={}&page={}"

# Defining an empty list of urls and first page of search
urls = []
page = 1

# Navigating to page 1 and define the maximum number of pages
url = url_template.format(search, page)
driver.get(url)

max_page_element = driver.find_elements_by_class_name("s-pagination-item.s-pagination-disabled")
max_page = []
for element in max_page_element:
    element = element.text.strip()
    max_page.append(element)
max_page = max_page[1]
max_page = int(max_page)

# Invalid strings to exclude from URLs
invalid_strings = ["offer-listing", "bestsellers", "ospublishing", "#customerReviews", ("s?k=" + search)]

# Loop over pages until we reach the maximum number of pages in the search
while page <= max_page:
    url = url_template.format(search, page)
    driver.get(url)
    
    # Find all the links on the page
    links = driver.find_elements_by_class_name('a-link-normal.s-underline-text.s-underline-link-text')

    # Iterate over the links and add them to the list
    for link in links:
        href = link.get_attribute("href")
        url_valid = True
        for invalid_string in invalid_strings:
            if invalid_string in href or search not in href:
                url_valid = False
                break
        if url_valid and href not in urls:
            urls.append(href)
            
    # Increase index of page
    page += 1

# create empty list for writing observations to
data = []

# Extracting information from each url
for url in urls:
    # Connecting to URL
    driver.get(url)
    # Finding the product ASIN by finding a match
    try:
        product_details_elements = driver.find_elements_by_class_name('a-size-base.prodDetAttrValue')
        product_details = []
        for element in product_details_elements:
            element = element.text.strip()
            if (len(element) == 10) & (element[0] == "B"):
                product_details.append(element)
        asin = product_details[0]
    except:
        asin = "NaN"
    
    # Finding the product name element and extracting its text
    try:
        name_element = driver.find_element_by_id('productTitle')
        name = name_element.text.strip()
    except:
        name = "NaN"
        
    # Finding the brand
    try:
        brand_element = driver.find_element_by_class_name('a-spacing-small.po-brand')
        brand = brand_element.text.strip()
    except:
        brand = "NaN"
    
    # Finding the list price
    try:
        list_price_element = driver.find_element_by_class_name('basisPrice')
        list_price = list_price_element.text.strip()
        list_price = list_price.split('$')[1]
    except:
        list_price = "NaN"
    
    # Finding the buy price
    try:
        buy_price_element = driver.find_element_by_class_name('priceToPay')
        buy_price = buy_price_element.text.strip()
        buy_price = buy_price.split()[0] + '.' + buy_price.split()[1]
        buy_price = buy_price[1:]
    try:
        buy_price_element = driver.find_element_by_class_name('a-color-price.a-text-bold')
        buy_price = buy_price_element.text.strip()
    except:
        buy_price = "NaN"
    
    
    # Finding the Prime exclusive price message and extract its text
    try:
        prime_price_element = driver.find_element_by_id('primeExclusivePricingMessage')
        prime_price = prime_price_element.text.strip()
        prime_price = prime_price.split('$')[1]
    except:
        prime_price = "NaN"
    
    # Finding the Average Rating
    try:
        rating_elements = driver.find_element_by_css_selector('[data-hook="rating-out-of-text"]')
        rating = rating_elements.text.strip()[:3]
    except:
        rating = "NaN"
    
    # Finding the count of ratings
    try:
        count_element = driver.find_element_by_id('acrCustomerReviewText')
        count = count_element.text.strip()
        count = count.split()[0]
    except:
        count = "NaN"
    
    # Finding the product description and extracting text elements as a list
    try:
        product_description_elements = driver.find_elements_by_id('feature-bullets')
        product_description = []
        for element in product_description_elements:
            element = element.text.strip()
            product_description.append(element)
        # Making product description a single string
        product_description = ''.join(product_description)
        # Removing special unicode characters that cannot be coded as ascii
        product_description = product_description.encode('ascii', 'ignore').decode('ascii')
    except:
        product_description = "NaN"
    
    # Finding the "ships from" label
    try:
        shipper_seller_elements = driver.find_elements_by_class_name('tabular-buybox-text.a-spacing-none')
        shipper_seller = []
        for element in shipper_seller_elements:
            element = element.text.strip()
            shipper_seller.append(element)
        shipper = shipper_seller[0]
        seller = shipper_seller[1]
    except:
        shipper = "NaN"
        seller = "NaN"
        
    # Finding the product category
    try:
        category_element = driver.find_element_by_id('wayfinding-breadcrumbs_feature_div')
        category = category_element.text.strip()
        category = category.split("›")
        category = category.pop()
    except:
        category = "NaN"
    
    data.append([asin, name, brand, list_price, buy_price, prime_price, rating, count, product_description, shipper, seller, category, url])
  
# Close the Firefox window
driver.quit()

# Create dataframe and export as csv
df = pd.DataFrame(data)   
df.columns = ['asin', 'name', 'brand', 'list_price', 'buy_price', 'prime_price', 'rating', 'count', 'product_description', 'shipper', 'seller', 'category', 'url']
df.to_csv((search + '.csv'), encoding='utf-8', index=False)  

This is a sample of EDA and data cleaning. Y'all's might have to look different to accomodate for different data but look through this for anything you might need. 

In [ ]:
#%% EDA and Cleaning
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

# Reading in data
os.chdir("C:/Users/cjconst/Documents/ECON 370")
microwaves = pd.read_csv("microwave.csv")

# Printing head of data
head = microwaves.head()
datatypes = microwaves.dtypes
print(datatypes)

# Converting all columns to string variables
microwaves = microwaves.astype(str).replace('nan', None)

# Removing commas from strings
microwaves['list_price'] = microwaves['list_price'].str.replace(',','')
microwaves['buy_price'] = microwaves['buy_price'].str.replace(',','')
microwaves['prime_price'] = microwaves['prime_price'].str.replace(',','')
microwaves['count'] = microwaves['count'].str.replace(',','')

# FIX "5 o" PROBLEM
microwaves['rating'] = microwaves['rating'].str.replace(' o','')

# Fix space on categories
microwaves['category'] = microwaves['category'].replace(r'\n','', regex=True) 

# Removing 'brand' from brand
microwaves['brand'] = microwaves['brand'].str.replace('Brand ', '')

# Creating a dictionary for column conversions
convert_dict = {'list_price': float,
                'buy_price': float,
                'prime_price': float,
                'rating': float,
                'count': float
                }
microwaves = microwaves.astype(convert_dict)
print(microwaves.dtypes)

# Summary info
microwaves.info()
microwaves.describe()

# Find unique values of brand, shipper, seller, and category
microwaves['brand'].unique()
microwaves["shipper"].unique()
microwaves["seller"].unique()
microwaves["category"].unique()

# Plot the unique values
microwaves['brand'].value_counts()[:20].plot(kind='bar')
microwaves['shipper'].value_counts()[:10].plot(kind='bar')
microwaves['seller'].value_counts()[:10].plot(kind='bar')
microwaves['category'].value_counts()[:10].plot(kind='bar')

# Only keeping actual microwaves
filter_ = microwaves['category'].isin(["Countertop Microwave Ovens", 'Over-the-Range Microwave Ovens', "Compact Microwave Ovens"])
microwaves = microwaves[filter_]
microwaves['category'] = microwaves['category'].astype("category")

# Reviewing nulls
microwaves.isnull().sum()

# Fixing the null ASINs
microwaves = microwaves.reset_index(drop=True)
for idx, x in enumerate(microwaves["asin"]):
    if x == None:      
        s = re.findall('dp/B.*/', microwaves["url"][idx])
        microwaves["asin"][idx] = s[0][3:13]
        
# Defining NA counts as zero
microwaves['count'].replace(np.nan,'0',inplace = True)
microwaves['count'] = microwaves['count'].astype(int)

# Defining NA ratings as mean of ratings
microwaves['rating'].replace(np.nan, microwaves['rating'].mean(),inplace = True)
microwaves['rating'] = microwaves['rating'].astype(float)
        
# Defining discount columns
microwaves["discount"] = (microwaves["list_price"] - microwaves["buy_price"])/microwaves["list_price"]
microwaves["prime_discount"] = (microwaves["buy_price"] - microwaves["prime_price"])/microwaves["buy_price"]

# Defining NA discounts as zero
microwaves['discount'].replace(np.nan, '0',inplace = True)
microwaves['discount'] = microwaves['discount'].astype(float)

# Defining NA brands as None
microwaves['brand'].replace(np.nan,'None',inplace = True)

# Defining Shipped by Amazon and Sold by Amazon Columns
def amazon(val):
    if val in ['Amazon', 'Amazon.com', 'Amazon Global Store UK']:
        return 1 
    else:
        return 0

microwaves["shipper"] = microwaves["shipper"].apply(amazon)
microwaves["seller"] = microwaves["seller"].apply(amazon)

# Dropping list and prime prices
microwaves = microwaves.drop(["list_price","prime_price"],axis=1)
microwaves = microwaves.reset_index()

# Renaming columns
microwaves.rename(columns={'buy_price':'price','index':'search_order'}, inplace = True)
microwaves["search_order"] = microwaves["search_order"].astype(int)

# Group means
microwaves.groupby(['brand'])['price'].mean()
microwaves.groupby(['shipper'])['price'].mean()
microwaves.groupby(['seller'])['price'].mean()
microwaves.groupby(['category'])['price'].mean()

# Scatter plot visualizations
plt.scatter(microwaves["search_order"], microwaves["price"], alpha=0.5)
plt.scatter(microwaves["rating"], microwaves["price"], alpha=0.5)
plt.scatter(np.log(microwaves["count"]), microwaves["price"], alpha=0.5)
plt.scatter(microwaves["discount"], microwaves["price"], alpha=0.5)

# Create a log price variable
microwaves["logprice"] = np.log(microwaves["price"])

# Writing csv of clean product
microwaves.to_csv('microwaves_clean.csv',index=False) 

This block is all of my modeling stuff. This should be pretty much 1:1 for all of us.

In [ ]:
#%% Creating dataframe with just prices and titles
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

os.chdir("C:/Users/cjconst/Documents/ECON 370")
microwaves = pd.read_csv("microwaves_clean.csv")

microwaves.head()

from sklearn.feature_extraction.text import TfidfVectorizer

# with smooth td-idf

tf_idf_vec_smooth = TfidfVectorizer(use_idf=True,  
                        smooth_idf=True,  
                        ngram_range=(2,2),
                        stop_words='english',
                        min_df=3)
 
 
tf_idf = tf_idf_vec_smooth.fit_transform(microwaves["name"])
 
tf_idf = pd.DataFrame(tf_idf.toarray(),columns=tf_idf_vec_smooth.get_feature_names_out())

tf_idf_scores = pd.DataFrame(tf_idf.mean())

# remerging data
microwaves = microwaves[["logprice","search_order","brand","rating","count","shipper","seller","category","discount"]]
microwaves = microwaves.merge(tf_idf, left_index=True, right_index=True)
h=microwaves.head()

# lasso regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import train_test_split, RepeatedKFold, KFold, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# Defining dummies for brand and category
microwaves = microwaves.merge(pd.get_dummies(microwaves[["brand","category"]]), left_index=True, right_index=True)
microwaves = microwaves.drop(columns=["brand","category"])

# Drop NAs and scale variables
microwaves = microwaves.dropna()
scaler = StandardScaler()
microwaves_scaled = scaler.fit_transform(microwaves)
microwaves_scaled = pd.DataFrame(data=microwaves_scaled,columns=microwaves.columns)

# Defining X and Y
Y = microwaves_scaled["logprice"].values
X = microwaves_scaled.drop(columns="logprice").values

# Train and test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=123)

# Describe data
Y_test.mean()
Y_test.std()
Y_test.min()
Y_test.max()

# Correlation matrix
corr_matrix = microwaves_scaled.corr()
sns.heatmap(corr_matrix)
plt.show()


#%% LASSO Function Definitions
# - From Mate Pocs: https://towardsdatascience.com/hyperparameter-tuning-in-lasso-and-ridge-regressions-70a4b158ae6d

def regmodel_param_plot(
    validation_score, train_score, alphas_to_try, chosen_alpha,
    scoring, model_name, test_score = None, filename = None):
    
    plt.figure(figsize = (8,8))
    sns.lineplot(y = validation_score, x = alphas_to_try, 
                 label = 'validation_data')
    sns.lineplot(y = train_score, x = alphas_to_try, 
                 label = 'training_data')
    plt.axvline(x=chosen_alpha, linestyle='--')
    if test_score is not None:
        sns.lineplot(y = test_score, x = alphas_to_try, 
                     label = 'test_data')
    plt.xlabel('alpha_parameter')
    plt.ylabel(scoring)
    plt.title(model_name + ' Regularisation')
    plt.legend()
    if filename is not None:
        plt.savefig(str(filename) + ".png")
    plt.show()
    
def regmodel_param_test(
    alphas_to_try, X, y, cv, scoring = 'r2', 
    model_name = 'LASSO', X_test = None, y_test = None, 
    draw_plot = False, filename = None):
    
    validation_scores = []
    train_scores = []
    results_list = []
    if X_test is not None:
        test_scores = []
        scorer = get_scorer(scoring)
    else:
        test_scores = None

    for curr_alpha in alphas_to_try:
        
        if model_name == 'LASSO':
            regmodel = Lasso(alpha = curr_alpha)
        elif model_name == 'Ridge':
            regmodel = Ridge(alpha = curr_alpha)
        else:
            return None

        results = cross_validate(
            regmodel, X, y, scoring=scoring, cv=cv, 
            return_train_score = True)

        validation_scores.append(np.mean(results['test_score']))
        train_scores.append(np.mean(results['train_score']))
        results_list.append(results)

        if X_test is not None:
            regmodel.fit(X,y)
            y_pred = regmodel.predict(X_test)
            test_scores.append(scorer(regmodel, X_test, y_test))
    
    chosen_alpha_id = np.argmax(validation_scores)
    chosen_alpha = alphas_to_try[chosen_alpha_id]
    max_validation_score = np.max(validation_scores)
    if X_test is not None:
        test_score_at_chosen_alpha = test_scores[chosen_alpha_id]
    else:
        test_score_at_chosen_alpha = None
        
    if draw_plot:
        regmodel_param_plot(
            validation_scores, train_scores, alphas_to_try, chosen_alpha, 
            scoring, model_name, test_scores, filename)
    
    return chosen_alpha, max_validation_score, test_score_at_chosen_alpha

#%% Regular Linear Regression (For Baseline)

lin = LinearRegression()

lin_fit = lin.fit(X_train, Y_train)

# finding mean squared error of model
lin_preds = lin_fit.predict(X_test)
lin_mse = mean_squared_error(Y_test, lin_preds).round(3)
print(f"The mean squared error for the linear regression model is {lin_mse}")

c=lin_fit.coef_

lin_errors = Y_test - lin_preds

plt.hist(lin_errors)

#%% CV LASSO Grid Search - First Pass
from sklearn.metrics import get_scorer

# defining a cross validation object
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# defining array of alphas from 0-1 with step of 0.1
alphas = np.linspace(0,1,11) 

# running the functions
chosen_alpha, max_validation_score, test_score_at_chosen_alpha = \
    regmodel_param_test(
        alphas, X_train, Y_train, 
        cv, scoring = 'r2', model_name = 'LASSO', 
        X_test = X_test, y_test = Y_test, 
        draw_plot = True, filename = 'lasso_wide_search')
    
print("Chosen alpha: %.5f" % \
    chosen_alpha)
print("Validation score: %.5f" % \
    max_validation_score)
print("Test score at chosen alpha: %.5f" % \
    test_score_at_chosen_alpha)
    
#%% CV LASSO Grid Search - Second Pass
# defining a cross validation object
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# defining array of alphas from 0-1 with step of 0.1
alphas = np.linspace(0,0.2,21) 

# running the functions
chosen_alpha, max_validation_score, test_score_at_chosen_alpha = \
    regmodel_param_test(
        alphas, X_train, Y_train, 
        cv, scoring = 'r2', model_name = 'LASSO', 
        X_test = X_test, y_test = Y_test, 
        draw_plot = True, filename = 'lasso_wide_search')
    
print("Chosen alpha: %.5f" % \
    chosen_alpha)
print("Validation score: %.5f" % \
    max_validation_score)
print("Test score at chosen alpha: %.5f" % \
    test_score_at_chosen_alpha)
    
#%% CV LASSO Grid Search - Third Pass
# defining a cross validation object
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# defining array of alphas from 0-1 with step of 0.1
alphas = np.linspace(0.003,0.1,21) 

# running the functions
chosen_alpha, max_validation_score, test_score_at_chosen_alpha = \
    regmodel_param_test(
        alphas, X_train, Y_train, 
        cv, scoring = 'r2', model_name = 'LASSO', 
        X_test = X_test, y_test = Y_test, 
        draw_plot = True, filename = 'lasso_wide_search')
    
print("Chosen alpha: %.5f" % \
    chosen_alpha)
print("Validation score: %.5f" % \
    max_validation_score)
print("Test score at chosen alpha: %.5f" % \
    test_score_at_chosen_alpha)

#%% Getting Coefficients from Final Lasso Model
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error

model = Lasso(alpha=chosen_alpha)

fit = model.fit(X_train, Y_train)

cdf = pd.DataFrame(fit.coef_, microwaves.columns[1:], columns=['Coefficients'])
print(cdf)

cdf["AbsCoef"] = abs(cdf["Coefficients"])

# finding mean squared error of model
preds = fit.predict(X_test)
lasso_mse = mean_squared_error(Y_test, preds).round(3)
print(f"The mean squared error for the lasso model is {lasso_mse}")

selected_vars = cdf[cdf["Coefficients"]>0]

lasso_vars = list(selected_vars.index.values)
lasso_vars.append("logprice")

microwaves_lassoed = microwaves[lasso_vars]

Y = microwaves_lassoed["logprice"]
X = microwaves_lassoed.drop(columns="logprice")

# remove space from column names
microwaves_lassoed.columns = microwaves_lassoed.columns.str.replace('_', '')
microwaves_lassoed.columns = microwaves_lassoed.columns.str.replace(' ', '_')
microwaves_lassoed.columns = microwaves_lassoed.columns.str.replace(',', '')
microwaves_lassoed.columns = microwaves_lassoed.columns.str.replace('.', '')
microwaves_lassoed.columns = microwaves_lassoed.columns.str.replace('-', '_')

microwaves_lassoed = microwaves_lassoed.rename(columns={'1700_watts':'watts_1700','30_cu':'cu_30'})

count = 1
xs = ""
for i in microwaves_lassoed.columns[:41]:
    if count == len(microwaves_lassoed.columns[:41]):
        xs = xs + i
    else:
        xs = xs + i + " + "
        count += 1
        
formula = 'logprice ~ ' + xs
        
mod1 = smf.ols(formula = formula, data=microwaves_lassoed).fit()
print(mod1.summary())

#%% RandomForest model
from sklearn.ensemble import RandomForestRegressor

tree = RandomForestRegressor()

tree_fit = tree.fit(X_train, Y_train)

# finding mean squared error of model
tree_preds = tree_fit.predict(X_test)
rf_mse = mean_squared_error(Y_test, tree_preds).round(3)
print(f"The mean squared error for the random forest model is {rf_mse}")

feature_names = [f"feature {i}" for i in range(X.shape[1])]

rf_importance = pd.DataFrame(tree_fit.feature_importances_, microwaves.columns[1:], columns=['Importance']).sort_values(by="Importance",ascending=False)
len(rf_importance[rf_importance['Importance'] > 0])

features = list(rf_importance.index)[0:20]
importances = rf_importance.values.reshape(len(rf_importance))[0:20]
 
# creating the bar plot
plt.barh(features, importances, color ='green',
        height = 0.4)
 
plt.ylabel("Features")
plt.xlabel("Mean decrease in impurity")
plt.title("Top twenty feature importances using MDI")
plt.gca().invert_yaxis() 
#plt.xticks(rotation=90)
plt.show()

#%% Boosted Regression Tree
import xgboost as xgb

# eta is equivalent to learning rate
# max_depth is the max tree depth
# subsample is proportion of sample used randomly for each tree
# colsample_bytree is the proportion of columns randomly selected to be in each tree
# alpha is regularization penalty (like lasso)
boost = xgb.XGBRegressor(n_estimators=1000, max_depth = 5, eta = 0.1, subsample = 1, colsample_bytree = 0.91, alpha = 0)

boost_fit = boost.fit(X_train, Y_train)

print(boost.importance_type)

# finding mean squared error of model
boost_preds = boost_fit.predict(X_test)
boost_mse = mean_squared_error(Y_test, boost_preds).round(3)
print(f"The mean squared error for the boosted regression model is {boost_mse}")

boost_importance = pd.DataFrame(boost_fit.feature_importances_, microwaves.columns[1:], columns=['Importance']).sort_values(by="Importance",ascending=False)
len(boost_importance[boost_importance['Importance'] > 0])
len(boost_importance)

boost_features = list(boost_importance.index)[0:20]
boost_importances = boost_importance.values.reshape(len(boost_importance))[0:20]

# creating the bar plot
plt.barh(boost_features, boost_importances, color ='green',
        height = 0.4)
 
plt.ylabel("Features")
plt.xlabel("Gain")
plt.title("Top twenty feature importances using Gain")
plt.gca().invert_yaxis() 
#plt.xticks(rotation=90)
plt.show()

mse = pd.DataFrame([lin_mse, lasso_mse, rf_mse, boost_mse],["Linear Regression - BoW","LASSO - BoW","RandomForest - BoW","Gradient Boosted Tree - BoW"])
